In [ ]:
# !pip install numpy==1.26.3 && pip install pandas==2.2.0 && pip install torch==2.1.2 && pip install transformers==4.37.0 && pip install datasets==2.1.0 && pip install evaluate==0.4.1 && pip install seqeval==1.2.2 && pip install peft==0.8.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=e1ad49b1c022abe84e7f53072a0ca5044d6f15828901cf296fee7a54df6f969a
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
# from google.colab import drive
# drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# installing packages
import numpy as np
import pandas as pd
import torch

from datasets import load_dataset
from transformers import AutoTokenizer, TokenClassificationPipeline
from transformers.pipelines import AggregationStrategy
from peft import (
    PeftModel,
    get_peft_model,
    LoraConfig,
    TaskType
)

In [ ]:
# Set the maximum length of tokenized sequences and stride for overlapping tokens
WINDOW_LENGTH = 512
STRIDE = 256



In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cpu


In [ ]:
# loading dataset and doing augumentation
dataset = load_dataset("HaniehPoostchi/persian_ner")

def tile_dataset(example, reps):

    example['tokens'] *= reps
    example['ner_tags'] *= reps


    return example

dataset = dataset.map(tile_dataset, fn_kwargs={"reps": 50})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/5121 [00:00<?, ?ex/s]

  0%|          | 0/2560 [00:00<?, ?ex/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 5121
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 2560
    })
})

In [ ]:
dataset['train'][0]

{'tokens': ['به',
  'عنوان',
  'مثال',
  'وقتی',
  'نشریات',
  'مدافع',
  'اصول',
  'و',
  'ارزشها',
  'و',
  'منادی',
  'انقلاب',
  'و',
  'اسلام',
  'در',
  'بالاترین',
  'درجه',
  '،',
  'اولین',
  'و',
  'درشت\u200cترین',
  'تیتر',
  'نشریه',
  'خود',
  'را',
  'در',
  'صدر',
  'صفحه',
  'نخستین',
  '،',
  'به',
  'تکذیب',
  'اظهارات',
  'و',
  'نظریات',
  'مشاور',
  'رئیس\u200cجمهور',
  'با',
  'همین',
  'ترکیب',
  'عبارتی',
  'و',
  'البته',
  'از',
  'قول',
  'دیگران',
  'اختصاص',
  'می\u200cدهند',
  '،',
  'آیا',
  'در',
  'موارد',
  'مشابه',
  'نیز',
  'هر',
  'گاه',
  'خبر',
  'تکذیب',
  'متوجه',
  'و',
  'معطوف',
  'به',
  'شخصی',
  'باشد',
  'كه',
  'در',
  'زمره',
  'مشاوران',
  'يك',
  'مقام',
  'بلندمرتبه\u200cی',
  'دیگر',
  'است',
  '،',
  'خبر',
  'را',
  'عینا',
  'به',
  'همین',
  'درشتی',
  'و',
  'با',
  'همین',
  'ترکیب',
  'عبارتی',
  'در',
  'صدر',
  'صفحه',
  'نخست',
  'به',
  'چاپ',
  'می\u200cرساند',
  'و',
  'در',
  'آن',
  'مورد',
  'هم',
  'به',
  'جای',


In [ ]:
# list of dataset labels
label_list = dataset["train"].features[f"ner_tags"].feature.names

In [ ]:
label_list

['O',
 'I-event',
 'I-fac',
 'I-loc',
 'I-org',
 'I-pers',
 'I-pro',
 'B-event',
 'B-fac',
 'B-loc',
 'B-org',
 'B-pers',
 'B-pro']

In [ ]:
llm_name = "HooshvareLab/bert-base-parsbert-armanner-uncased"
# Define tokenizer configuration options
tok_kwargs = {
    'model_max_length': WINDOW_LENGTH,       # Maximum length of tokenized input sequences
    'truncation': True,                      # Truncate sequences longer than max length
    'padding': "max_length",                 # Pad sequences to the maximum length
    'stride': STRIDE,                        # Stride size for splitting long sequences
    'return_overflowing_tokens': True,       # Return overflowing token sequences
    'is_split_into_words': True              # Input is split into words (list of words)
}
# loading tokenizer with our custom argument
tokenizer = AutoTokenizer.from_pretrained(
    llm_name,
    **tok_kwargs
    )

In [ ]:
from transformers import PreTrainedTokenizer, PreTrainedTokenizerFast
from datasets.arrow_dataset import Batch
# from datasets import Batch
from transformers.tokenization_utils_base import BatchEncoding

In [ ]:
def tokenize_and_align_labels(
        instances: dict,
        llm_tok: PreTrainedTokenizer | PreTrainedTokenizerFast
        ) -> BatchEncoding:
    """Divides tokens of input data into sliding windows and aligns labels.

    Upon tokenizing the input data into the defined window lengths, the labels
    are rearranged to match the shape of the tokenized text. This is done by
    iterating over every sliding window that was just created and creating
    a separate list of labels for each sliding window. This function follows
    the logic of the `tokenize_and_align_labels` function from the huggingface
    tutorial (https://huggingface.co/docs/transformers/tasks/token_classification)
    and makes the said function compatible to the use case of passing large text
    data into an LLM through sliding windows.

    Args:
        instances: One or more row(s) of a huggingface dataset.
        llm_tok: Initialized LLM tokenizer.
    Returns:
        Tokenized instances with aligned labels.
    """
    tokenized_instances = llm_tok(
        instances["tokens"],
        truncation=True,
        padding="max_length",
        max_length=WINDOW_LENGTH,
        stride=STRIDE,
        return_overflowing_tokens=True,
        is_split_into_words=True
    )

    labels_input = instances["ner_tags"]
    labels = []
    n_windows = len(tokenized_instances['input_ids'])
    for window_i, instance_i in zip(
        range(n_windows),
        tokenized_instances['overflow_to_sample_mapping']
    ):
        window_labels = []
        word_indexes = tokenized_instances.word_ids(window_i)
        previous_word_idx = None

        for word_idx in word_indexes:  # Set the special tokens to -100.
            if word_idx is None:
                window_labels.append(-100)
            # Only label the first token of a given word.
            # Assign label to right sublist if input overflows.
            elif word_idx != previous_word_idx:
                window_labels.append(labels_input[instance_i][word_idx])
            else:
                window_labels.append(-100)
            previous_word_idx = word_idx

        labels.append(window_labels)

    tokenized_instances["labels"] = labels

    return tokenized_instances


In [ ]:
import os
import numpy as np
import torch
import evaluate
from transformers import (
    AutoModelForTokenClassification,
    DataCollatorForTokenClassification,
    TrainingArguments,
    Trainer
)

# Extract the list of label names from the dataset for the NER task
label_list = dataset["train"].features["ner_tags"].feature.names

# Tokenize and align labels for the entire dataset
dataset_tokenized = dataset.map(
    tokenize_and_align_labels, fn_kwargs={"llm_tok": tokenizer}, batched=True,
    remove_columns=dataset["train"].column_names
)

# Initialize the data collator for token classification to handle padding and alignment
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

# Load the seqeval metric for evaluating token classification tasks
seqeval = evaluate.load("seqeval")

# Define a function to compute evaluation metrics
def compute_metrics(p):
    predictions, labels = p
    # Get the predicted class for each token
    predictions = np.argmax(predictions, axis=2)

    # Remove special tokens and convert IDs to label names for predictions and true labels
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    # Compute the evaluation metrics using seqeval
    results = seqeval.compute(
        predictions=true_predictions,
        references=true_labels
    )
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

# Create mappings between label IDs and label names
id2label = {i: label for i, label in enumerate(label_list)}
label2id = {label: i for i, label in id2label.items()}


  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [ ]:
dataset_tokenized

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'overflow_to_sample_mapping', 'labels'],
        num_rows: 31152
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'overflow_to_sample_mapping', 'labels'],
        num_rows: 15213
    })
})

In [ ]:
label2id

{'O': 0,
 'I-event': 1,
 'I-fac': 2,
 'I-loc': 3,
 'I-org': 4,
 'I-pers': 5,
 'I-pro': 6,
 'B-event': 7,
 'B-fac': 8,
 'B-loc': 9,
 'B-org': 10,
 'B-pers': 11,
 'B-pro': 12}

In [ ]:
id2label

{0: 'O',
 1: 'I-event',
 2: 'I-fac',
 3: 'I-loc',
 4: 'I-org',
 5: 'I-pers',
 6: 'I-pro',
 7: 'B-event',
 8: 'B-fac',
 9: 'B-loc',
 10: 'B-org',
 11: 'B-pers',
 12: 'B-pro'}

In [ ]:
# loading the base model
model = AutoModelForTokenClassification.from_pretrained(
    llm_name,
    num_labels=13,
    id2label=id2label,
    label2id=label2id,
    # ignore_mismatched_sizes=True
)

Some weights of the model checkpoint at HooshvareLab/bert-base-parsbert-armanner-uncased were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from peft import (
    get_peft_model,  # Function to get the model with parameter-efficient fine-tuning (PEFT)
    LoraConfig,     # Class for configuring LoRA (Low-Rank Adaptation)
    TaskType        # Enum for specifying the task type
)

# Define the target modules in the model that will be adapted using LoRA
tar_modules = ["key", "query", "value"]

# Configure LoRA for the token classification task
peft_config = LoraConfig(
    task_type=TaskType.TOKEN_CLS,  # Specify that the task is token classification
    inference_mode=False,          # Set to False for training mode
    r=16,                          # Low-rank dimension for adaptation
    lora_alpha=16,                 # Scaling factor for the low-rank adaptation
    lora_dropout=0.1,              # Dropout rate for the low-rank adaptation
    bias="all",                    # Apply LoRA to all bias terms in the model
    target_modules=tar_modules,    # Modules in the model to apply LoRA
    modules_to_save=["classifier"] # Save these modules during training
)

# Apply the PEFT configuration to the model to create a LoRA-adapted model
lora_model = get_peft_model(model, peft_config)

# Print the trainable parameters of the LoRA-adapted model
lora_model.print_trainable_parameters()


trainable params: 996,877 || all params: 163,155,482 || trainable%: 0.610998164315435


In [ ]:
# Move the LoRA-adapted model to the specified device (e.g., GPU or CPU)
lora_model.to(device)

# Set up training arguments for the Trainer
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/DM-PROJECT/Model/model_training",  # Directory to save model checkpoints and outputs
    learning_rate=2e-5,          # Learning rate for training
    per_device_train_batch_size=4,  # Batch size for training
    per_device_eval_batch_size=16,  # Batch size for evaluation
    num_train_epochs=3,         # Number of training epochs
    weight_decay=0.01,          # Weight decay for regularization
    fp16=True,                  # Enable 16-bit floating point precision for faster training
    evaluation_strategy="epoch",  # Evaluate the model at the end of each epoch
    save_strategy="epoch",      # Save the model at the end of each epoch
    load_best_model_at_end=True, # Load the best model found during training at the end
    push_to_hub=False,          # Disable pushing the model to the Hugging Face Hub
    report_to="none"            # Disable reporting to any external services like W&B
)

# Initialize the Trainer with the LoRA-adapted model and training arguments
trainer = Trainer(
    model=lora_model,                # Model to be trained
    args=training_args,              # Training arguments
    train_dataset=dataset_tokenized["train"],  # Training dataset
    eval_dataset=dataset_tokenized["test"],    # Evaluation dataset
    tokenizer=tokenizer,             # Tokenizer used for preprocessing
    data_collator=data_collator,     # Data collator to handle padding and batching
    compute_metrics=compute_metrics, # Function to compute evaluation metrics
)

# Start the training process
trainer.train()

# Save the fine-tuned LoRA-adapted model to the specified directory
trainer.save_model('/content/drive/MyDrive/DM-PROJECT/Model/lora_model_fintetuned')


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:447: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.009500,0.019773,0.934556,0.957253,0.945768,0.994569
2,0.006000,0.018124,0.937854,0.959725,0.948664,0.995004
3,0.004100,0.018319,0.939045,0.959264,0.949047,0.995077


Checkpoint destination directory /content/drive/MyDrive/DM-PROJECT/Model/model_training/checkpoint-7788 already exists and is non-empty.Saving will proceed but saved results may be invalid.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, us

In [ ]:
from transformers import AutoModelForTokenClassification
from peft import PeftModel

# Instantiate the base model for token classification
model = AutoModelForTokenClassification.from_pretrained(
    llm_name,            # Pre-trained model name
    num_labels=13,       # Number of labels for token classification
    id2label=id2label,   # Dictionary mapping label IDs to label names
    label2id=label2id,   # Dictionary mapping label names to label IDs
    # ignore_mismatched_sizes=True  # Uncomment if the model's head size doesn't match the number of labels
)

# Load the fine-tuned LoRA model from the specified directory
inference_model = PeftModel.from_pretrained(model, "/content/drive/MyDrive/DM-PROJECT/Model/lora_model_fintetuned")

# Merge the LoRA adaptations into the base model and unload them
merged_model = inference_model.merge_and_unload()

# Save the fully fine-tuned model to the specified directory
merged_model.save_pretrained("/content/drive/MyDrive/DM-PROJECT/Model/full_finetuned_model")


Some weights of the model checkpoint at HooshvareLab/bert-base-parsbert-armanner-uncased were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
!pip install tensorflow hazm


ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_i

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
tokenizer

BertTokenizerFast(name_or_path='HooshvareLab/bert-base-parsbert-armanner-uncased', vocab_size=100000, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [ ]:
import tensorflow as tf
from transformers import AutoConfig, AutoTokenizer, TFAutoModelForTokenClassification
import hazm

# Translation dictionary for NER labels
arman_translate = {
    "B-event": "رویداد",
    "B-fac": "امکانات",
    "B-loc": "موقعیت",
    "B-org": "سازمان",
    "B-pers": "شخص",
    "B-pro": "محصول",
    "I-event": "رویداد",
    "I-fac": "امکانات",
    "I-loc": "موقعیت",
    "I-org": "سازمان",
    "I-pers": "شخص",
    "I-pro": "محصول",
    "O": None
}

# Normalizer instance for Persian text
normalizer = hazm.Normalizer()

def cleanize(text):
    """Normalize and clean the text using hazm.Normalizer"""
    return normalizer.normalize(text)

def parsbert_ner_load_model(model_name):
    """Load the fine-tuned model for NER"""
    try:
        config = AutoConfig.from_pretrained(model_name)

        model = TFAutoModelForTokenClassification.from_pretrained(model_name)
        labels = list(config.id2label.values())  # Get labels from model config

        return model, labels
    except Exception as e:
        print(f"Error loading model: {e}")
        return None, None, None

def parsbert_ner(texts, model_name, label_translate):
    """Perform NER on input texts using the specified model"""
    model, labels = parsbert_ner_load_model(model_name)

    if not model or not tokenizer or not labels:
        return 'Error: Failed to load model.'

    output_predictions = []
    for sequence in texts:
        sequence = cleanize(sequence)
        tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(sequence)))
        inputs = tokenizer.encode(sequence, return_tensors="tf")
        outputs = model(inputs)[0]
        predictions = tf.argmax(outputs, axis=2)

        # Process predictions to match with translated labels
        predictions = [(token, label_translate[labels[prediction]]) for token, prediction in zip(tokens, predictions[0].numpy())]

        output_predictions.append(predictions)

    return output_predictions

# Example input texts
texts = [
    "سازمان ملل در نیویورک واقع شده است.",
    "جشنواره بین‌المللی فیلم کن",
    "شرکت اپل محصولات خوبی تولید می‌کند."
]

model_name = "/content/drive/MyDrive/DM-PROJECT/Model/full_finetuned_model"

# Perform NER
res = parsbert_ner(texts, model_name, arman_translate)
print(res)


All PyTorch model weights were used when initializing TFBertForTokenClassification.

All the weights of TFBertForTokenClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForTokenClassification for predictions without further training.


[[('[CLS]', None), ('سازمان', 'سازمان'), ('ملل', 'سازمان'), ('در', None), ('نیویورک', 'موقعیت'), ('واقعشده', None), ('است', None), ('.', None), ('[SEP]', None)], [('[CLS]', None), ('جشنواره', 'رویداد'), ('بینالمللی', 'رویداد'), ('فیلم', 'رویداد'), ('کن', 'رویداد'), ('[SEP]', None)], [('[CLS]', None), ('شرکت', 'سازمان'), ('اپل', 'سازمان'), ('محصولات', None), ('خوبی', None), ('تولید', None), ('میکند', None), ('.', None), ('[SEP]', None)]]


In [ ]:
# Example input texts
texts = [
    "سازمان ملل در نیویورک واقع شده است.",
    "جشنواره بین‌المللی فیلم کن",
    "شرکت اپل محصولات خوبی تولید می‌کند."
]

model_name = "/content/drive/MyDrive/DM-PROJECT/Model/full_finetuned_model"

# Perform NER
res = parsbert_ner(texts, model_name, arman_translate)
print(res)

All PyTorch model weights were used when initializing TFBertForTokenClassification.

All the weights of TFBertForTokenClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForTokenClassification for predictions without further training.


[[('[CLS]', None), ('سازمان', 'سازمان'), ('ملل', 'سازمان'), ('در', None), ('نیویورک', 'موقعیت'), ('واقعشده', None), ('است', None), ('.', None), ('[SEP]', None)], [('[CLS]', None), ('جشنواره', 'رویداد'), ('بینالمللی', 'رویداد'), ('فیلم', 'رویداد'), ('کن', 'رویداد'), ('[SEP]', None)], [('[CLS]', None), ('شرکت', 'سازمان'), ('اپل', 'سازمان'), ('محصولات', None), ('خوبی', None), ('تولید', None), ('میکند', None), ('.', None), ('[SEP]', None)]]


In [ ]:
# Example input texts
texts = [
    "تهران پایتخت ایران است.",
    "شرکت مایکروسافت نرم‌افزارهای مختلفی تولید می‌کند.",
    "کتاب‌های جلال آل احمد بسیار مشهور هستند."
]

model_name = "/content/drive/MyDrive/DM-PROJECT/Model/full_finetuned_model"

# Perform NER
res = parsbert_ner(texts, model_name, arman_translate)
print(res)


All PyTorch model weights were used when initializing TFBertForTokenClassification.

All the weights of TFBertForTokenClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForTokenClassification for predictions without further training.


[[('[CLS]', None), ('تهران', 'موقعیت'), ('پایتخت', None), ('ایران', 'موقعیت'), ('است', None), ('.', None), ('[SEP]', None)], [('[CLS]', None), ('شرکت', 'سازمان'), ('مایکروسافت', 'سازمان'), ('نرمافزارهای', None), ('مختلفی', None), ('تولید', None), ('میکند', None), ('.', None), ('[SEP]', None)], [('[CLS]', 'محصول'), ('کتابهای', None), ('جلال', 'محصول'), ('ال', 'محصول'), ('احمد', 'محصول'), ('بسیار', None), ('مشهور', None), ('هستند', None), ('.', None), ('[SEP]', None)]]


In [ ]:
# Example input texts
texts = [
    "دانشگاه تهران یکی از معتبرترین دانشگاه‌های ایران است.",
    "فیلم جدید تارانتینو در جشنواره کن جایزه گرفت.",
    "گوگل بزرگترین شرکت فناوری اطلاعات در جهان است."
]

model_name = "/content/drive/MyDrive/DM-PROJECT/Model/full_finetuned_model"

# Perform NER
res = parsbert_ner(texts, model_name, arman_translate)
print(res)


All PyTorch model weights were used when initializing TFBertForTokenClassification.

All the weights of TFBertForTokenClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForTokenClassification for predictions without further training.


[[('[CLS]', None), ('دانشگاه', 'امکانات'), ('تهران', 'امکانات'), ('یکی', None), ('از', None), ('معتبرترین', None), ('دانشگاههای', None), ('ایران', 'موقعیت'), ('است', None), ('.', None), ('[SEP]', None)], [('[CLS]', None), ('فیلم', None), ('جدید', None), ('تارانتینو', 'شخص'), ('در', None), ('جشنواره', 'رویداد'), ('کن', 'رویداد'), ('جایزه', None), ('گرفت', None), ('.', None), ('[SEP]', None)], [('[CLS]', None), ('گوگل', 'سازمان'), ('بزرگترین', None), ('شرکت', 'سازمان'), ('فناوری', 'سازمان'), ('اطلاعات', 'سازمان'), ('در', None), ('جهان', None), ('است', None), ('.', None), ('[SEP]', None)]]


In [ ]:
# Example input texts
texts = [
    "شیراز به خاطر باغ‌های زیبایش معروف است.",
    "استیو جابز یکی از بنیان‌گذاران اپل بود.",
    "فیلم های هالیوود در سراسر جهان شناخته شده‌اند."
]

model_name = "/content/drive/MyDrive/DM-PROJECT/Model/full_finetuned_model"

# Perform NER
res = parsbert_ner(texts, model_name, arman_translate)
print(res)


All PyTorch model weights were used when initializing TFBertForTokenClassification.

All the weights of TFBertForTokenClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForTokenClassification for predictions without further training.


[[('[CLS]', None), ('شیراز', 'موقعیت'), ('به', None), ('خاطر', None), ('باغهای', None), ('زیبایش', None), ('معروف', None), ('است', None), ('.', None), ('[SEP]', None)], [('[CLS]', None), ('استیو', 'شخص'), ('جابز', 'شخص'), ('یکی', None), ('از', None), ('بنیانگذاران', None), ('اپل', 'سازمان'), ('بود', None), ('.', None), ('[SEP]', None)], [('[CLS]', None), ('فیلمهای', None), ('هالیوود', 'سازمان'), ('در', None), ('سراسر', None), ('جهان', None), ('شناخته', None), ('شدهاند', None), ('.', None), ('[SEP]', None)]]


In [ ]:
# test like how bert is being tested

from transformers import pipeline

# assert that we use fast version
assert tokenizer.is_fast

# instantiate pipeline
classifier_1 = pipeline(
    "token-classification",
    model="/content/drive/MyDrive/DM-PROJECT/Model/full_finetuned_model",
    tokenizer=tokenizer,
    stride=STRIDE,  # additional parameter
    aggregation_strategy='max'
)

# standard token classification pipeline expects untokenized input
sentence = " ".join(dataset['test']['tokens'][0])
# retokenize sentence for subsequent print statements
tokenized_sentence = tokenizer(
    sentence,
    truncation=True,
    padding="max_length",
    max_length=WINDOW_LENGTH,
    stride=STRIDE,
    return_overflowing_tokens=True,
    is_split_into_words=False
)
# get text length (i.e. number of characters) of first window
text_window_1 = " ".join(
    [tokenizer.decode(tkn) for tkn in tokenized_sentence['input_ids'][0]]
)
n_words = len(dataset['test']['tokens'][0])

print(f"total number of words: {n_words}, total length of text: {len(sentence)}")
# returns: total number of words: 1350, total length of text: 6899
print(f"length of text in first window: {len(text_window_1)}")
# returns: length of text in first window: 2542

output = classifier_1(sentence)

last_token = output[-1]
print(f"last word tagged at text position: {last_token['start']}-{last_token['end']}")
# returns: last word tagged at text position: 6862-6869

total number of words: 12000, total length of text: 54199
length of text in first window: 2766
last word tagged at text position: 54134-54141


In [ ]:
print(output)

[{'entity_group': 'loc', 'score': 0.9514407, 'word': 'ایران', 'start': 44, 'end': 49}, {'entity_group': 'loc', 'score': 0.999585, 'word': 'تهران', 'start': 122, 'end': 127}, {'entity_group': 'loc', 'score': 0.9071438, 'word': 'استونی در حوضه بالتیک', 'start': 237, 'end': 258}, {'entity_group': 'loc', 'score': 0.99932456, 'word': 'ایران', 'start': 322, 'end': 327}, {'entity_group': 'loc', 'score': 0.94742936, 'word': 'افغانستان', 'start': 379, 'end': 388}, {'entity_group': 'loc', 'score': 0.9995498, 'word': 'خراسان', 'start': 421, 'end': 427}, {'entity_group': 'loc', 'score': 0.9990287, 'word': 'المان', 'start': 504, 'end': 509}, {'entity_group': 'loc', 'score': 0.9964587, 'word': 'نايورو', 'start': 690, 'end': 696}, {'entity_group': 'loc', 'score': 0.9973424, 'word': 'اقیانوسیه', 'start': 700, 'end': 709}, {'entity_group': 'org', 'score': 0.96391433, 'word': 'ایرانی', 'start': 730, 'end': 736}, {'entity_group': 'loc', 'score': 0.99915195, 'word': 'یوگسلاوی', 'start': 960, 'end': 968}, 

In [ ]:
# test like how bert is being tested

from transformers import pipeline

# assert that we use fast version
assert tokenizer.is_fast

# instantiate pipeline
classifier_1 = pipeline(
    "token-classification",
    model="/content/drive/MyDrive/DM-PROJECT/Model/full_finetuned_model",
    tokenizer=tokenizer,
    stride=STRIDE,  # additional parameter
    aggregation_strategy='max'
)

# standard token classification pipeline expects untokenized input
sentence = " ".join(dataset['test']['tokens'][1])
# retokenize sentence for subsequent print statements
tokenized_sentence = tokenizer(
    sentence,
    truncation=True,
    padding="max_length",
    max_length=WINDOW_LENGTH,
    stride=STRIDE,
    return_overflowing_tokens=True,
    is_split_into_words=False
)
# get text length (i.e. number of characters) of first window
text_window_1 = " ".join(
    [tokenizer.decode(tkn) for tkn in tokenized_sentence['input_ids'][0]]
)
n_words = len(dataset['test']['tokens'][1])

print(f"total number of words: {n_words}, total length of text: {len(sentence)}")
# returns: total number of words: 1350, total length of text: 6899
print(f"length of text in first window: {len(text_window_1)}")
# returns: length of text in first window: 2542

output = classifier_1(sentence)

last_token = output[-1]
print(f"last word tagged at text position: {last_token['start']}-{last_token['end']}")
# returns: last word tagged at text position: 6862-6869

total number of words: 1850, total length of text: 8499
length of text in first window: 2493
last word tagged at text position: 8457-8466


In [ ]:
print(output)

[{'entity_group': 'pers', 'score': 0.9994205, 'word': 'طاهایی', 'start': 0, 'end': 6}, {'entity_group': 'event', 'score': 0.9641587, 'word': 'هفته دولت', 'start': 127, 'end': 136}, {'entity_group': 'pers', 'score': 0.9995455, 'word': 'طاهایی', 'start': 170, 'end': 176}, {'entity_group': 'event', 'score': 0.9674402, 'word': 'هفته دولت', 'start': 297, 'end': 306}, {'entity_group': 'pers', 'score': 0.9996008, 'word': 'طاهایی', 'start': 340, 'end': 346}, {'entity_group': 'event', 'score': 0.9674823, 'word': 'هفته دولت', 'start': 467, 'end': 476}, {'entity_group': 'pers', 'score': 0.999595, 'word': 'طاهایی', 'start': 510, 'end': 516}, {'entity_group': 'event', 'score': 0.9693706, 'word': 'هفته دولت', 'start': 637, 'end': 646}, {'entity_group': 'pers', 'score': 0.9996178, 'word': 'طاهایی', 'start': 680, 'end': 686}, {'entity_group': 'event', 'score': 0.96270025, 'word': 'هفته دولت', 'start': 807, 'end': 816}, {'entity_group': 'pers', 'score': 0.9995658, 'word': 'طاهایی', 'start': 850, 'end':

In [ ]:
from transformers import pipeline

def test_ner_model(model_path, tokenizer, texts):
    # Ensure the tokenizer is the fast version
    assert tokenizer.is_fast

    # Instantiate the pipeline
    ner_pipeline = pipeline(
        "token-classification",
        model=model_path,
        tokenizer=tokenizer,
        stride=STRIDE,
        aggregation_strategy='max'
    )

    # Process each text and print the results
    for text in texts:
        tokenized_sentence = tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=WINDOW_LENGTH,
            stride=STRIDE,
            return_overflowing_tokens=True,
            is_split_into_words=False
        )

        # Decode tokenized input to ensure it is in the expected format
        tokenized_input = tokenizer.decode(tokenized_sentence['input_ids'][0], skip_special_tokens=True)

        n_words = len(text.split())
        text_length = len(text)

        # Get the length of text in the first window
        text_window_1 = " ".join([tokenizer.decode(tkn) for tkn in tokenized_sentence['input_ids'][0]])
        first_window_length = len(text_window_1)
        # Get NER results using the tokenized input
        ner_results = ner_pipeline(tokenized_input)


        # Print general information
        print(f"Text: {text}")
        print(f"Total number of words: {n_words}, Total length of text: {text_length}")
        print(f"Length of text in first window: {first_window_length}")

        # Print the structure of ner_results to understand the keys
        print(f"NER results structure for text: {text}")
        for entity in ner_results:
            print(entity)

        if ner_results:
            last_token = ner_results[-1]
            print(f"Last word tagged at text position: {last_token['start']}-{last_token['end']}")

    return ner_pipeline

# Example usage
model_path = "/content/drive/MyDrive/DM-PROJECT/Model/full_finetuned_model"
texts = [
    "شیراز به خاطر باغ‌های زیبایش معروف است.",
    "استیو جابز یکی از بنیان‌گذاران اپل بود.",
    "فیلم های هالیوود در سراسر جهان شناخته شده‌اند."
]

# Assuming tokenizer is already loaded
# tokenizer = AutoTokenizer.from_pretrained(model_path)

test_ner_model(model_path, tokenizer, texts)


Text: شیراز به خاطر باغ‌های زیبایش معروف است.
Total number of words: 7, Total length of text: 39
Length of text in first window: 3063
NER results structure for text: شیراز به خاطر باغ‌های زیبایش معروف است.
{'entity_group': 'loc', 'score': 0.9940607, 'word': 'شیراز', 'start': 0, 'end': 5}
Last word tagged at text position: 0-5
Text: استیو جابز یکی از بنیان‌گذاران اپل بود.
Total number of words: 7, Total length of text: 39
Length of text in first window: 3063
NER results structure for text: استیو جابز یکی از بنیان‌گذاران اپل بود.
{'entity_group': 'pers', 'score': 0.9989834, 'word': 'استیو جابز', 'start': 0, 'end': 10}
{'entity_group': 'org', 'score': 0.92679596, 'word': 'اپل', 'start': 30, 'end': 33}
Last word tagged at text position: 30-33
Text: فیلم های هالیوود در سراسر جهان شناخته شده‌اند.
Total number of words: 8, Total length of text: 46
Length of text in first window: 3064
NER results structure for text: فیلم های هالیوود در سراسر جهان شناخته شده‌اند.
{'entity_group': 'pro', 'score'

In [ ]:
llm_name

'HooshvareLab/bert-base-parsbert-armanner-uncased'

# results before fine-tuning

In [ ]:
# test like how bert is being tested

from transformers import pipeline

# assert that we use fast version
assert tokenizer.is_fast

# instantiate pipeline
classifier_1 = pipeline(
    "token-classification",
    model=llm_name,
    tokenizer=tokenizer,
    stride=STRIDE,  # additional parameter
    aggregation_strategy='max'
)

# standard token classification pipeline expects untokenized input
sentence = " ".join(dataset['test']['tokens'][0])
# retokenize sentence for subsequent print statements
tokenized_sentence = tokenizer(
    sentence,
    truncation=True,
    padding="max_length",
    max_length=WINDOW_LENGTH,
    stride=STRIDE,
    return_overflowing_tokens=True,
    is_split_into_words=False
)
# get text length (i.e. number of characters) of first window
text_window_1 = " ".join(
    [tokenizer.decode(tkn) for tkn in tokenized_sentence['input_ids'][0]]
)
n_words = len(dataset['test']['tokens'][0])

print(f"total number of words: {n_words}, total length of text: {len(sentence)}")
# returns: total number of words: 1350, total length of text: 6899
print(f"length of text in first window: {len(text_window_1)}")
# returns: length of text in first window: 2542

output = classifier_1(sentence)

last_token = output[-1]
print(f"last word tagged at text position: {last_token['start']}-{last_token['end']}")
# returns: last word tagged at text position: 6862-6869

Some weights of the model checkpoint at HooshvareLab/bert-base-parsbert-armanner-uncased were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


total number of words: 12000, total length of text: 54199
length of text in first window: 2766
last word tagged at text position: 54134-54141


In [ ]:
print(output)

[{'entity_group': 'loc', 'score': 0.76303405, 'word': 'ایران', 'start': 44, 'end': 49}, {'entity_group': 'loc', 'score': 0.9984346, 'word': 'تهران', 'start': 122, 'end': 127}, {'entity_group': 'loc', 'score': 0.8174063, 'word': 'استونی در حوضه بالتیک', 'start': 237, 'end': 258}, {'entity_group': 'loc', 'score': 0.99822396, 'word': 'ایران', 'start': 322, 'end': 327}, {'entity_group': 'loc', 'score': 0.91785246, 'word': 'افغانستان', 'start': 379, 'end': 388}, {'entity_group': 'loc', 'score': 0.9991259, 'word': 'خراسان', 'start': 421, 'end': 427}, {'entity_group': 'loc', 'score': 0.9989454, 'word': 'المان', 'start': 504, 'end': 509}, {'entity_group': 'loc', 'score': 0.9824692, 'word': 'نايورو', 'start': 690, 'end': 696}, {'entity_group': 'loc', 'score': 0.99696785, 'word': 'اقیانوسیه', 'start': 700, 'end': 709}, {'entity_group': 'org', 'score': 0.98741895, 'word': 'ایرانی', 'start': 730, 'end': 736}, {'entity_group': 'loc', 'score': 0.996816, 'word': 'یوگسلاوی', 'start': 960, 'end': 968},